<a href="https://colab.research.google.com/github/WilliamAshbee/minigan-catalyst/blob/master/LSTMTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://github.com/bhpfelix/PyTorch-Time-Series-Classification-Benchmarks/blob/master/MaterialRecognitionModels.ipynb
import shutil, os, csv, itertools, glob

import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from sklearn.metrics import confusion_matrix

import pandas as pd
import pickle as pk

cuda = torch.cuda.is_available()


In [ ]:
## Multilayer LSTM based classifier taking in 200 dimensional fixed time series inputs
class LSTMClassifier(nn.Module):

    def __init__(self, in_dim, hidden_dim, num_layers, dropout, bidirectional, num_classes, batch_size):
        super(LSTMClassifier, self).__init__()
        self.arch = 'lstm'
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_dir = 2 if bidirectional else 1
        self.num_layers = num_layers

        self.lstm = nn.LSTM(
                input_size=in_dim,
                hidden_size=hidden_dim,
                num_layers=num_layers,
                dropout=dropout,
                bidirectional=bidirectional
            )

        self.hidden2label = nn.Sequential(
            nn.Linear(hidden_dim*self.num_dir, hidden_dim),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(hidden_dim, num_classes),
            nn.Sigmoid()
        )

        # self.hidden = self.init_hidden()

    def init_hidden(self):
        if cuda:
            h0 = Variable(torch.zeros(self.num_layers*self.num_dir, self.batch_size, self.hidden_dim).cuda())
            c0 = Variable(torch.zeros(self.num_layers*self.num_dir, self.batch_size, self.hidden_dim).cuda())
        else:
            h0 = Variable(torch.zeros(self.num_layers*self.num_dir, self.batch_size, self.hidden_dim))
            c0 = Variable(torch.zeros(self.num_layers*self.num_dir, self.batch_size, self.hidden_dim))
        return (h0, c0)

    def forward(self, x): # x is (batch_size, 1, 200), permute to (200, batch_size, 1)
        x = x.permute(2, 0, 1)
        # See: https://discuss.pytorch.org/t/solved-why-we-need-to-detach-variable-which-contains-hidden-representation/1426/2
        lstm_out, (h, c) = self.lstm(x, self.init_hidden())
        y  = self.hidden2label(lstm_out[-1])
        return y


In [ ]:
def get_models(): # tuples of (batch_size, model)
    return [
         LSTMClassifier(
            in_dim=2,
            hidden_dim=120,
            num_layers=3,
            dropout=0.8,
            bidirectional=True,
            num_classes=1,#bce loss for discriminator
            batch_size=256
        )
    ]


In [ ]:
model = get_models()[0]

In [ ]:
model

LSTMClassifier(
  (lstm): LSTM(2, 120, num_layers=3, dropout=0.8, bidirectional=True)
  (hidden2label): Sequential(
    (0): Linear(in_features=240, out_features=120, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=120, out_features=120, bias=True)
    (4): ReLU(inplace=True)
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=120, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [ ]:
x = torch.ones(256,2,200).float().cuda()
model = model.cuda()

In [ ]:
model(x).shape


torch.Size([256, 1])

In [ ]:
# Dummy Dataset for testing purpose only
class DummyDataset(Dataset):
    """Time Series dataset."""

    def __init__(self, numclasses=15):
        self.numclasses = numclasses

    def __len__(self):
        return 512

    def __getitem__(self, idx):
        data = None
        y = None
        if torch.rand(1)[0] > .5:
          data = torch.ones(2,200).float()
          y = float(1)
        else:
          data = torch.zeros(2,200).float()
          y = float(0)
          
        return data, y


In [ ]:
data = DummyDataset(numclasses=2)
D = model.cuda()
train_loader = torch.utils.data.DataLoader(
    data,
    batch_size=256, shuffle=True)
D_optimizer = optim.Adam(D.parameters(), lr=.0001, betas=(0.5, 0.999))


In [ ]:
bce = torch.nn.BCELoss()

In [ ]:
for epoch in range(100):
  for x_, y_ in train_loader:
    y_ = y_.unsqueeze(dim = 1).float().cuda()
    x_ = x_.cuda()

    D_result = D(x_)
    #print(D_result)
    #break
    #print(D_result)
    D_train_loss = bce(D_result, y_)
    print('D_train_loss', D_train_loss)
    D_train_loss.backward()
    D_optimizer.step()



D_train_loss tensor(0.6920, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6941, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6932, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6932, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6951, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6928, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6905, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6927, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6949, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6925, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6911, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6916, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_tr

In [ ]:
data1 = torch.ones(1,2,200).float()
y1 = torch.ones(1,1)
data0 = torch.zeros(1,2,200).float()
y0 = torch.zeros(1,1)
        

In [ ]:
for x, y in train_loader:
  x = x.cuda()
  print(torch.sum(model(x)>.5))
  print(torch.sum(y))

tensor(126, device='cuda:0')
tensor(126., dtype=torch.float64)
tensor(133, device='cuda:0')
tensor(133., dtype=torch.float64)


In [ ]:
# Dummy Dataset for testing purpose only
class DummyDataset2(Dataset):
    """Time Series dataset."""

    def __len__(self):
        return 512

    def __getitem__(self, idx):
        data = None
        y = None
        ind = [x for x in range(200)]
        ind = torch.tensor(ind).float()
        if torch.rand(1)[0] > .5:
          data = torch.ones(2,200).float()
          data[0,:] = torch.sin(ind) +torch.rand(1)[0]
          data[1,:] = torch.cos(ind) +torch.rand(1)[0]
          y = float(1)
        else:
          data = torch.ones(2,200).float()
          
          data[0,:] = torch.sin(ind*2.0) +torch.rand(1)[0] 
          data[1,:] = torch.cos(ind*2.0) +torch.rand(1)[0] 
          y = float(0)
          
        return data, y


In [ ]:
data = DummyDataset2()
D = get_models()[0].cuda()
train_loader = torch.utils.data.DataLoader(
    data,
    batch_size=256, shuffle=True)
D_optimizer = optim.Adam(D.parameters(), lr=.0001, betas=(0.5, 0.999))


In [ ]:
for epoch in range(100):
  for x_, y_ in train_loader:
    y_ = y_.unsqueeze(dim = 1).float().cuda()
    x_ = x_.cuda()

    D_result = D(x_)
    #print(D_result)
    #break
    #print(D_result)
    D_train_loss = bce(D_result, y_)
    print('D_train_loss', D_train_loss)
    D_train_loss.backward()
    D_optimizer.step()

D_train_loss tensor(0.6925, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6944, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6929, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6936, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6946, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6940, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6941, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6967, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6937, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6933, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6938, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_train_loss tensor(0.6941, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
D_tr

In [ ]:
#https://github.com/bhpfelix/PyTorch-Time-Series-Classification-Benchmarks/blob/master/MaterialRecognitionModels.ipynb